#  Tutorial 01 – Running a Basic Simulation

This tutorial walks through running a basic Pywr-DRB simulation using a YAML configuration file and `sample_run.py`. You’ll create a model JSON using `ModelBuilder`, run a simulation using `pywrdrb.Model`, and save the output to a standardized HDF5 file.

This is a great place to start if you're new to the Pywr-DRB model.


## Step 1 – Initialize Model with `ModelBuilder`

We start by creating a model configuration (`model.json`) using `pywrdrb.ModelBuilder`. This file defines the structure of the model: start and end dates, inflow dataset, node relationships, and operational constraints.

### Built-in Inflow Type Options

Pywr-DRB supports a range of inflow datasets that represent historic, modeled, and hybrid hydrologic conditions:

- `'nhmv10'` → National Hydrologic Model v1.0 (raw)
- `'nhmv10_withObsScaled'` → NHM v1.0 scaled with observed flows
- `'nwmv21'` → National Water Model v2.1 (raw)
- `'nwmv21_withObsScaled'` → NWM v2.1 scaled with observed flows
- `'pub_nhmv10_BC_withObsScaled'` → Public reconstruction of NHM v1.0 with bias correction and observed scaling
- `'wrf1960s_calib_nlcd2016'` → WRF downscaled inflows calibrated to 1960s climate with NLCD 2016 land cover
- `'wrf2050s_calib_nlcd2016'` → WRF downscaled inflows calibrated to 2050s climate with NLCD 2016 land cover
- `'wrfaorc_calib_nlcd2016'` → WRF historical (AORC forcing) calibrated inflows with NLCD 2016
- `'wrfaorc_withObsScaled'` → WRF historical AORC inflows scaled with observed flows
- `'_scaled_inflows'` → Experimental scaled inflow set
- `'_hydro_model_flow_output'` → Experimental hydrologic model output flow set

These inflow types allow testing a wide variety of hydrologic scenarios — from historic reconstructions to future climate-driven projections.

---

### Allowable Simulation Periods

Available periods depend on the dataset:
- `'nhmv10'`, `'nhmv10_withObsScaled'`: ~1983–2016
- `'nwmv21'`, `'nwmv21_withObsScaled'`: ~1983–2020
- `'pub_nhmv10_BC_withObsScaled'`: ~1945–2023
- `'wrf1960s'`, `'wrf2050s'`, `'wrfaorc'`: modeled periods specific to scenario (check dataset metadata)

⚠ Always check the dataset metadata or file coverage before setting `start_date` and `end_date` in your simulation.


We will use the `'nhmv10_withObsScaled'` inflow dataset as an example.



In [8]:
import pywrdrb

mb = pywrdrb.ModelBuilder(
    inflow_type='nhmv10_withObsScaled',
    start_date="1983-10-01",
    end_date="2016-12-31"
)
mb.make_model()

model_filename = "./model.json"
mb.write_model(model_filename)


## Step 2 – Load the Model

Now that the model JSON file has been created, we can load it into memory using `pywrdrb.Model.load()`. This initializes all the Pywr nodes, parameters, and connections defined in the JSON file.


In [2]:
model = pywrdrb.Model.load(model_filename)


## Step 3 – Attach an Output Recorder

Next, we attach an `OutputRecorder` to the model. This customized recorder tracks key outputs during the simulation and writes them to an `.hdf5` file.

The output file will store time series data like reservoir storage, streamflow, and releases.

### Optional Arguments for OutputRecorder

The `pywrdrb.OutputRecorder` records and saves model simulation data to an HDF5 file.  
It wraps the base `pywr.Recorder` class and is optimized for efficiency while tracking all key model variables.

Key arguments:
- `model` → the `pywrdrb.Model` instance to record from (required)
- `output_filename` → name of the output HDF5 file (required)
- `nodes` → optional list of `pywr.core.Node` objects to record (if `None`, all named nodes are recorded)
- `parameters` → optional list of `pywr.core.Parameter` objects to record (if `None`, all named parameters are recorded)


In [3]:
output_filename = "./pywrdrb_output.hdf5"
recorder = pywrdrb.OutputRecorder(model, output_filename)


## Step 4 – Run the Simulation

Now we run the model. This may take a few seconds depending on your machine. The simulation uses the configuration and input data to simulate reservoir operations and streamflows across the Delaware River Basin.


In [4]:
stats = model.run()


c:\Users\ms3654\Desktop\Research\Delaware\DRB_development\venv\Lib\site-packages\pywr\dataframe_tools.py:127: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  new_df = new_df.resample(target_index.freq).agg(resample_func)
C:\Users\ms3654\AppData\Local\Temp\ipykernel_6956\1470591929.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  stats = model.run()


## Step 5 – Confirm the Output File Was Created

After running the simulation, check that the HDF5 output file was successfully created. This file will be used in the next tutorial to analyze results.


In [7]:
import os

assert os.path.exists(output_filename), "Simulation output not found."
print(" Simulation completed and output file saved:", output_filename)


 Simulation completed and output file saved: ./pywrdrb_output.hdf5
